In [1]:
import sys
sys.path.append("../../current_work/")
from pandas.core.indexing import maybe_convert_ix
import PredictionManager as pm
import backtest_class as backTest
import yFinance_class as yfHelper
import pandas as pd

## Data'nın indirilmesi

In [2]:
# use "period" instead of start/end
        # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
        # (optional, default is '1mo')
        #period = "ytd",

# fetch data by interval (including intraday if period < 60 days)
        # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
        # (optional, default is '1d')
        #interval = "1m",

In [3]:
userInput = "SASA.IS"
period = '1y'
interval = '1d'

In [4]:
yData = yfHelper.yFinance(userInput.upper(), period, interval).getData()

[*********************100%***********************]  1 of 1 completed


In [5]:
yData.iloc[0]

Date         2021-01-14 00:00:00
Open                   17.400364
High                   17.993221
Low                    16.940901
Close                  17.237328
Adj Close              17.237328
Volume                  23535028
Name: 0, dtype: object

In [6]:
yData.iloc[-1]

Date         2022-01-14 00:00:00
Open                        53.0
High                   54.650002
Low                        52.75
Close                  53.650002
Adj Close              53.650002
Volume                  16438281
Name: 250, dtype: object

In [7]:
df = yData.copy()
df = df.set_index(pd.DatetimeIndex(df['Date'].values))

In [8]:
col_list = df.columns.to_list()

In [9]:
col_list

['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']

In [10]:
len(col_list)

7

In [11]:
col_list[1:len(col_list)-1]

['Open', 'High', 'Low', 'Close', 'Adj Close']

In [12]:
df

,Date,Open,High,Low,Close,Adj Close,Volume
2021-01-14,2021-01-14,17.400364,17.993221,16.940901,17.237328,17.237328,23535028
2021-01-15,2021-01-15,17.192863,17.637506,17.089113,17.622686,17.622686,9466830
2021-01-18,2021-01-18,17.652328,18.141436,17.652328,17.993221,17.993221,13715209
2021-01-19,2021-01-19,18.156258,19.015902,17.859829,18.645365,18.645365,24703051
2021-01-20,2021-01-20,18.704651,19.549473,18.511972,19.267864,19.267864,22130800
...,...,...,...,...,...,...,...
2022-01-10,2022-01-10,55.650002,58.099998,55.200001,56.500000,56.500000,29600100
2022-01-11,2022-01-11,57.400002,57.599998,55.200001,55.349998,55.349998,18439456
2022-01-12,2022-01-12,56.450001,56.450001,54.299999,54.700001,54.700001,12400597
2022-01-13,2022-01-13,54.299999,55.200001,53.000000,53.099998,53.099998,12435985


In [13]:
type(df.iloc[2,1])

numpy.float64

In [14]:
for i in range(0,10):
    if (df.iloc[i,2] > 0):
        print("ok")
        print(df.iloc[i,2])

ok
17.993221282958984
ok
17.63750648498535
ok
18.141435623168945
ok
19.015901565551758
ok
19.54947280883789
ok
19.66804313659668
ok
19.001079559326172
ok
18.94179344177246
ok
19.801437377929688
ok
19.845901489257812


## RSI parametresinin çıkartılması

In [15]:
import pandas as pd
import numpy as np

In [16]:
close = yData.Close
ret = close.diff() #(n+1).gün - n.gün
up = []
down = [] 

for element in range(0, len(ret)):
    if (ret[element] < float(0)):
        up.append(0)
        down.append(ret[element])
    elif (ret[element] >= float(0)):
        up.append(ret[element])
        down.append(0)
        
upSeries = pd.Series(up)
downSeries = pd.Series(down).abs()

In [17]:
ret = ret.replace(np.nan, 0)
ret

0      0.000000
1      0.385359
2      0.370535
3      0.652143
4      0.622499
         ...   
246    1.299999
247   -1.150002
248   -0.649998
249   -1.600002
250    0.550003
Name: Close, Length: 251, dtype: float64

In [18]:
lookback = 7
upEwm = upSeries.ewm(com = lookback -1, adjust = False).mean()
downEwm = downSeries.ewm(com = lookback -1, adjust = False).mean()

In [19]:
rs = upEwm / downEwm
rsi = 100 - (100 / (1 + rs))

In [20]:
close = close.drop(index = close.index[0])
temp_df = pd.DataFrame(rsi).rename(columns={0 : 'rsi'}).set_index(close.index)


In [21]:
print("toplam gün sayısı:", len(yData))
print("rsi veri gün sayısı:", len(rsi))

toplam gün sayısı: 251
rsi veri gün sayısı: 250


In [22]:
#bu işlemi yapmamızın sebebi hesaplamalarda ilk iki gün 
#RSI değerinin 100 çıkması
#o yüzden ilk 2 günü pass geçiyoruz hatta 3 günü de geçebiliriz.
yData["RSI_14"] = temp_df[2:]

## Algoritmanın geliştirilmesi

### 1- RSI ile alım satım

In [23]:
lower_band = np.float(30)
upper_band = np.float(70)
signal = 0 #sinyal parametresi sattıysak -1 aldıysak 1 olacak.
rsi_signal = []
buy_price = [] #alım yaptıgımızda alım fiyatını ekledigimiz liste
sell_price = []#satış yaptıgımızda satış fiyatını ekledigimiz liste
rsi = yData.RSI_14
prices = yData.Close
def wait_for_signal(buyList, sellList, signalList):
    buyList.append(np.nan)
    sellList.append(np.nan)
    signalList.append(0)
    
def fill_buy_signal(buyList,buyPrice, sellList, signalList):
    buyList.append(buyPrice)
    sellList.append(np.nan)
    signalList.append(1)
    
def fill_sell_signal(buyList, sellList, sellPrice, signalList):
    buyList.append(np.nan)
    sellList.append(sellPrice)
    signalList.append(-1)
    
for element in range(1, len(rsi)):
    if ((rsi[element - 1] > lower_band) and (rsi[element] < lower_band)):
        #eğer rsi değeri 30'un üstünden 30'un altına geçiş yaptıysa
        #alım yapmak istiyoruz
        #ancak iki kere alım yapamayacağımızdan sinyal kontrolü yap.
        if (signal != 1):
            fill_buy_signal(buy_price, prices[element], sell_price, rsi_signal)
            signal = 1
        else:
            wait_for_signal(buy_price, sell_price, rsi_signal)
            
    elif((rsi[element - 1] < upper_band) and (rsi[element] > upper_band)):
        #eğer rsi değeri 70'in altındaysa ve 70'in üstüne geçiş yaptıysa
        #satış yapmak istiyoruz
        #ancak iki kere satış yapamacağımızdan sinyal kontrolü yap
        if (signal != -1):
            fill_sell_signal(buy_price, sell_price, prices[element], rsi_signal)
            signal = -1
        else:
            wait_for_signal(buy_price, sell_price, rsi_signal)
            
    else:
        wait_for_signal(buy_price, sell_price, rsi_signal)
            
            
        

/Users/batuhanduyuler/opt/anaconda3/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  """Entry point for launching an IPython kernel.
/Users/batuhanduyuler/opt/anaconda3/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  


In [24]:
investmenValue = yData['Close'][0] * 5
firstInvestment = investmenValue
decisionFlag = False
stockNumber = round(investmenValue / yData['Close'][0])


for i in range(0, len(rsi) - 1):
    if (rsi_signal[i] != 0):
        if (decisionFlag == False):
            if(rsi_signal[i] == 1):
                decisionFlag = True
            elif(rsi_signal[i] == -1):
                decisionFlag = True
                investmenValue = yData["Close"][i+1] * stockNumber    
        else:
            if (rsi_signal[i] == 1):
                stockNumber = round(investmenValue / yData['Close'][i+1])
                investmenValue = 0
            elif (rsi_signal[i] == -1):
                investmenValue = yData['Close'][i+1] * stockNumber

                stockNumber = 0
      


if (investmenValue == 0.0):
    investmenValue = yData['Close'][len(yData) - 1] * stockNumber
    stockNumber = 0
    
print("% ",(investmenValue - firstInvestment) / firstInvestment * 100)
print(investmenValue - firstInvestment)

%  -3.233262412634536
-2.786640167236328


In [25]:
print("işlem yapılmasa durum: %",(yData.Close.iloc[-1] - yData.Close.iloc[0]) / yData.Close.iloc[0]*100)

işlem yapılmasa durum: % 211.24315118059283


In [26]:
alim_satim = 0
for i in rsi_signal:
    alim_satim += abs(i)
    
print("alım satım sayısı: ", alim_satim)
print("veri uzunlugu: ", len(yData))

alım satım sayısı:  3
veri uzunlugu:  251


### 2- Safe Sell RSI algoritması

In [27]:
lower_band = np.float(30)
upper_band = np.float(70)
signal = 0 #sinyal parametresi sattıysak -1 aldıysak 1 olacak.
rsi_signal = []
buy_price = [] #alım yaptıgımızda alım fiyatını ekledigimiz liste
sell_price = []#satış yaptıgımızda satış fiyatını ekledigimiz liste
rsi = yData.RSI_14
beklenenKar = 0.1
prices = yData.Close
lastBuyPrice = []
def wait_for_signal(buyList, sellList, signalList):
    buyList.append(np.nan)
    sellList.append(np.nan)
    signalList.append(0)
    
def fill_buy_signal(buyList,buyPrice, sellList, signalList):
    buyList.append(buyPrice)
    sellList.append(np.nan)
    signalList.append(1)
    
def fill_sell_signal(buyList, sellList, sellPrice, signalList):
    buyList.append(np.nan)
    sellList.append(sellPrice)
    signalList.append(-1)
    
for element in range(1, len(rsi)):
    if ((rsi[element - 1] > lower_band) and (rsi[element] < lower_band)):
        #eğer rsi değeri 30'un üstünden 30'un altına geçiş yaptıysa
        #alım yapmak istiyoruz
        #ancak iki kere alım yapamayacağımızdan sinyal kontrolü yap.
        if (signal != 1):
            fill_buy_signal(buy_price, prices[element], sell_price, rsi_signal)
            signal = 1
            lastBuyPrice.append(prices[element])
        else:
            #RSI alım sinyali verdi ama zaten alım yapmışız
            #fakat karımız %5 üzerindeyse satış yapmak istiyoruz.
            #o yüzden ilk önce fiyat düşüşte mi ona bakıyoruz
            if (prices[element - 1] > prices[element]):
                #eğer düşüşteyse aldığımız fiyattan yüksek mi
                #onu kontrol etmeliyiz.
                if (prices[element] > lastBuyPrice[-1]):
                    #eğer öyleyse karımızı hesaplamalıyız
                    lossPortion = (prices[element] - lastBuyPrice[-1]) / lastBuyPrice[-1]
                    if (lossPortion > beklenenKar ):
                        print(lossPortion)
                        fill_sell_signal(buy_price, sell_price, prices[element], rsi_signal)
                        signal = -1
                    else:
                        wait_for_signal(buy_price, sell_price, rsi_signal)
                else:
                    wait_for_signal(buy_price, sell_price, rsi_signal)
            else:
                wait_for_signal(buy_price, sell_price, rsi_signal)
            
    elif((rsi[element - 1] < upper_band) and (rsi[element] > upper_band)):
        #eğer rsi değeri 70'in altındaysa ve 70'in üstüne geçiş yaptıysa
        #satış yapmak istiyoruz
        #ancak iki kere satış yapamacağımızdan sinyal kontrolü yap
        if (signal != -1):
            fill_sell_signal(buy_price, sell_price, prices[element], rsi_signal)
            signal = -1
        else:
            wait_for_signal(buy_price, sell_price, rsi_signal)
         #satış pozisyonunda bir daha satış yapamayacağımızdan burayı pass geçiyoruz.   
    else:
        #eğer bu durumlardan biri yaşanmadıysa, karı kontrol edebiliriz.
        #ilk olarak sinyal alım pozisyonunda mı ona bakıyoruz.
        if (signal == 1):
            if (prices[element - 1] > prices[element]):
                #fiyat düşüştüyse tespit ettik.
                if (prices[element] > lastBuyPrice[-1]):
                    #karda olup olmadığımızı kontrol ettik
                    lossPortion = (prices[element] - lastBuyPrice[-1]) / lastBuyPrice[-1]
                    if (lossPortion > beklenenKar):
                        print(lossPortion)
                        fill_sell_signal(buy_price, sell_price, prices[element], rsi_signal)
                        signal = -1
                    else:
                        wait_for_signal(buy_price, sell_price, rsi_signal) 
                else:
                    wait_for_signal(buy_price, sell_price, rsi_signal)
            else:
                wait_for_signal(buy_price, sell_price, rsi_signal)
        else:
            wait_for_signal(buy_price, sell_price, rsi_signal)
            
        

/Users/batuhanduyuler/opt/anaconda3/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  """Entry point for launching an IPython kernel.
/Users/batuhanduyuler/opt/anaconda3/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  


In [28]:
alim_satim = 0
for i in rsi_signal:
    alim_satim += abs(i)
    
print("alım satım sayısı: ", alim_satim)
print("veri uzunlugu: ", len(yData))

alım satım sayısı:  3
veri uzunlugu:  251


In [29]:
len(rsi_signal)

250

In [30]:
new_data = yData.to_string(index = False)

In [31]:
print("% ",(investmenValue - firstInvestment) / firstInvestment * 100)
print(investmenValue - firstInvestment)

%  -3.233262412634536
-2.786640167236328


In [32]:
print("işlem yapılmasa durum: %",(yData.Close.iloc[-1] - yData.Close.iloc[0]) / yData.Close.iloc[0] * 100)

işlem yapılmasa durum: % 211.24315118059283


In [33]:
yData

,Date,Open,High,Low,Close,Adj Close,Volume,RSI_14
0,2021-01-14,17.400364,17.993221,16.940901,17.237328,17.237328,23535028,NaN
1,2021-01-15,17.192863,17.637506,17.089113,17.622686,17.622686,9466830,NaN
2,2021-01-18,17.652328,18.141436,17.652328,17.993221,17.993221,13715209,NaN
3,2021-01-19,18.156258,19.015902,17.859829,18.645365,18.645365,24703051,100.000000
4,2021-01-20,18.704651,19.549473,18.511972,19.267864,19.267864,22130800,100.000000
...,...,...,...,...,...,...,...,...
246,2022-01-10,55.650002,58.099998,55.200001,56.500000,56.500000,29600100,69.342211
247,2022-01-11,57.400002,57.599998,55.200001,55.349998,55.349998,18439456,60.651195
248,2022-01-12,56.450001,56.450001,54.299999,54.700001,54.700001,12400597,56.021152
249,2022-01-13,54.299999,55.200001,53.000000,53.099998,53.099998,12435985,45.947952


In [34]:
temp_df = pd.DataFrame(list(zip(yData.Date[1:].to_list(), yData.Close[1:].to_list(), rsi_signal)), columns = ["Date", "Close", "Signal"])

In [35]:
temp_df.Signal.unique()

array([ 0, -1,  1])